In [21]:
import requests
import datetime

def fetch_air_range(station_id, date_from, date_until):
    url = 'http://airviro.klab.ee/station/csv'
    data = {
        'filter[type]': 'INDICATOR',
        'filter[cancelSearch]': '',
        'filter[stationId]': station_id,
        'filter[dateFrom]': date_from,
        'filter[dateUntil]': date_until,
        'filter[submitHit]': '1',
        'filter[indicatorIds]': ''
    }
    response = requests.post(url, data)
    return response.text

def get_first_and_last_day_of_month(year, month):
    # Get the first day of the month
    first_day = datetime.date(year, month, 1)

    # Get the number of days in the month
    if month == 12:
        num_days = 31
    else:
        num_days = (datetime.date(year, month+1, 1) - datetime.timedelta(days=1)).day

    # Get the last day of the month
    last_day = datetime.date(year, month, num_days)

    return first_day, last_day

In [23]:
year_q = 2022
for month in range(1, 13):
    first_day, last_day = get_first_and_last_day_of_month(year_q, month)
    print(first_day, last_day)

    data = fetch_air_range(8, first_day, last_day)
    with open(f'air_data/air_{month + year_q}.csv', 'w', encoding='UTF-8') as f:
        f.write(data)


2022-01-01 2022-01-31
2022-02-01 2022-02-28
2022-03-01 2022-03-31
2022-04-01 2022-04-30
2022-05-01 2022-05-31
2022-06-01 2022-06-30
2022-07-01 2022-07-31
2022-08-01 2022-08-31
2022-09-01 2022-09-30
2022-10-01 2022-10-31
2022-11-01 2022-11-30
2022-12-01 2022-12-31


In [25]:
year_q = 2021
data = []
for month in range(1, 13):
    with open(f'air_data/air_{month}.csv', 'r') as f:
        if month == 1:
            header = f.readline()
            data.append(header)
        else:
            f.readline()
        data.extend(f.readlines())

with open('air_data/air_{year_q}.csv', 'w') as f:
    f.writelines(data)

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('air_data/air-2022-csv.sql')

df = pd.read_sql_query("SELECT * FROM air_2022", conn)
df['DateTime'] = pd.to_datetime(df['DateTime'])
df_day = df.groupby(df['DateTime'].dt.date).mean().reset_index()
df_month = df.groupby(df['DateTime'].dt.to_period('M')).mean().reset_index()

df_day.to_sql('air_2022_day', conn, if_exists='replace')
df_month.to_sql('air_2022_month', conn, if_exists='replace')

conn.close()

In [3]:
import requests
def fetch_electric(date_from, date_until):
    url = 'https://comptool.e.elering.ee/api/chartCSV'
    data = {
        'filter[mp1]': '38ZEE-04422581-Z',
        'filter[name1]': 'T%C3%B5nningi%20talu,%20Piirsalu%20k%C3%BCla,%20L%C3%A4%C3%A4ne-Nigula%20vald,%20L%C3%A4%C3%A4ne%20maakond,%2090902',
        'filter[kind1]': 'UsagePoint.Electricity',
        'filter[start]': date_from,
        'filter[end]': date_until,
        'filter[locale]': 'et-EE',
        'filter[addProduction]': False
    }
    response = requests.post(url, data)
    return response.text

data = fetch_electric(1648760400000, 1680296399000)
with open(f'el_data/elect_year.csv', 'w', encoding='UTF-8') as f:
    f.write(data)

In [2]:
import requests
def fetch_nord_pool(end_date):
    url = 'https://www.nordpoolgroup.com/api/marketdata/page/47'
    data1 = {
        'filter[currency]': ',EUR,EUR,EUR',
        'filter[endDate]': end_date
    }
    response2 = requests.post(url, data1)
    return response2.text

data2 = fetch_nord_pool(22-12-2022)
with open(f'el_data/nord_pool.csv', 'w', encoding='UTF-8') as f:
    f.write(data2)